**1) Write the ERC-20 Token Contracts**

1st i create 2 ERC-20 tokens:

**Token1.sol**

    // SPDX-License-Identifier: MIT
    pragma solidity ^0.8.19;

    import "@openzeppelin/contracts/token/ERC20/ERC20.sol";

    contract Token1 is ERC20 {
     constructor() ERC20("Token One", "TK1") {
        _mint(msg.sender, 1000000 * 10 ** decimals()); // Mint 1 million tokens
     }
    }

**Token2.sol**

    // SPDX-License-Identifier: MIT
    pragma solidity ^0.8.19;

    import "@openzeppelin/contracts/token/ERC20/ERC20.sol";

    contract Token2 is ERC20 {
     constructor() ERC20("Token Two", "TK2") {
        _mint(msg.sender, 1000000 * 10 ** decimals()); // Mint 1 million tokens
     }
    }

Now I Deploy Tokens

I Create a new file deploy.js inside the scripts folder:

    const hre = require("hardhat");

    async function main() {
      const [deployer] = await hre.ethers.getSigners();

      console.log("Deploying contracts with:", deployer.address);

      // Deploy Token1
      const Token1 = await hre.ethers.getContractFactory("Token1");
      const token1 = await Token1.deploy();
      await token1.deployed();
      console.log("Token1 deployed at:", token1.address);

      // Deploy Token2
      const Token2 = await hre.ethers.getContractFactory("Token2");
      const token2 = await Token2.deploy();
      await token2.deployed();
      console.log("Token2 deployed at:", token2.address);
    }

    main()
      .then(() => process.exit(0))
      .catch((error) => {
        console.error(error);
        process.exit(1);
      });


now i run this command to configure all things:

    *    npx hardhat run scripts/deploy.js --network sepolia


**2) Implement the Exchange Contract**

Now, let's create the Exchange.sol contract:

its solidity:

    // SPDX-License-Identifier: MIT
    pragma solidity ^0.8.19;

    import "@openzeppelin/contracts/token/ERC20/IERC20.sol";

    contract TokenExchange {
    address public owner;
    IERC20 public token1;
    IERC20 public token2;
    
    uint256 public rateToken1ToToken2; // How many token2 per token1
    uint256 public rateToken2ToToken1; // How many token1 per token2

    event Swap(address indexed user, address indexed tokenIn, address indexed tokenOut, uint256 amountIn, uint256 amountOut);
    event RateUpdated(uint256 newRateToken1ToToken2, uint256 newRateToken2ToToken1);

    modifier onlyOwner() {
        require(msg.sender == owner, "Not contract owner");
        _;
    }

    constructor(address _token1, address _token2, uint256 _rate1To2, uint256 _rate2To1) {
        owner = msg.sender;
        token1 = IERC20(_token1);
        token2 = IERC20(_token2);
        rateToken1ToToken2 = _rate1To2;
        rateToken2ToToken1 = _rate2To1;
    }

    function updateRate(uint256 _rate1To2, uint256 _rate2To1) external onlyOwner {
        rateToken1ToToken2 = _rate1To2;
        rateToken2ToToken1 = _rate2To1;
        emit RateUpdated(_rate1To2, _rate2To1);
    }

    function exchangeToken1(uint256 amount) external {
        require(amount > 0, "Amount must be greater than zero");

        uint256 amountOut = (amount * rateToken1ToToken2) / 1e18; // Fixed precision
        require(token2.balanceOf(address(this)) >= amountOut, "Not enough liquidity");

        require(token1.transferFrom(msg.sender, address(this), amount), "Token1 transfer failed");
        require(token2.transfer(msg.sender, amountOut), "Token2 transfer failed");

        emit Swap(msg.sender, address(token1), address(token2), amount, amountOut);
    }

    function exchangeToken2(uint256 amount) external {
        require(amount > 0, "Amount must be greater than zero");

        uint256 amountOut = (amount * rateToken2ToToken1) / 1e18; // Fixed precision
        require(token1.balanceOf(address(this)) >= amountOut, "Not enough liquidity");

        require(token2.transferFrom(msg.sender, address(this), amount), "Token2 transfer failed");
        require(token1.transfer(msg.sender, amountOut), "Token1 transfer failed");

        emit Swap(msg.sender, address(token2), address(token1), amount, amountOut);
     }
    }



**3) Deploy the Exchange Contract**

Now I Create a new deploy-exchange.js file in the scripts folder.


    const hre = require("hardhat");

    async function main() {
      const [deployer] = await hre.ethers.getSigners();

      console.log("Deploying contracts with:", deployer.address);

      // Token addresses from previous deployment
      const token1Address = "0xYourToken1Address";
      const token2Address = "0xYourToken2Address";

      const Exchange = await hre.ethers.getContractFactory("TokenExchange");
      const exchange = await Exchange.deploy(token1Address, token2Address, 2e18, 5e17); // Example rates
      await exchange.deployed();

      console.log("Exchange deployed at:", exchange.address);
    }

    main()
      .then(() => process.exit(0))
      .catch((error) => {
        console.error(error);
        process.exit(1);
      });

i run this command to configure this file:

    *    npx hardhat run scripts/deploy-exchange.js --network sepolia




**4) Interacting with the Contract**

Approve tokens before swapping:

    *    const token1 = await ethers.getContractAt("IERC20","0xYourToken1Address");

    *    await token1.approve("0xExchangeAddress", ethers.utils.parseUnits("100", 18));

Now Swap Token1 for Token2:

    *    const exchange = await ethers.getContractAt("TokenExchange", "0xExchangeAddress");
    *    await exchange.exchangeToken1(ethers.utils.parseUnits("10", 18));

Now Swap Token2 for Token1:

    *    await exchange.exchangeToken2(ethers.utils.parseUnits("5", 18));

Frontend Integration:

I create a React.js + Ethers.js frontend for interacting with MetaMask.

